# Run Qwen2.5-VL-32B on Amazon SageMaker AI with SGLang

❗This notebook works well on `ml.g5.xlarge` instance with 100GB of disk size and `PyTorch 2.2.0 Python 3.10 CPU optimized kernel` from **SageMaker Studio Classic** or `Python3 kernel` from **JupyterLab**

Note that SageMaker provides [pre-built SageMaker AI Docker images](https://docs.aws.amazon.com/sagemaker/latest/dg/pre-built-containers-frameworks-deep-learning.html) that can help you quickly start with the model inference on SageMaker. It also allows you to [bring your own Docker container](https://docs.aws.amazon.com/sagemaker/latest/dg/adapt-inference-container.html) and use it inside SageMaker AI for training and inference. To be compatible with SageMaker AI, your container must have the following characteristics:

- Your container must have a web server listening on port `8080`.
- Your container must accept POST requests to the `/invocations` and `/ping` real-time endpoints.

In this notebook, we'll demonstrate how to adapt the [SGLang](https://github.com/sgl-project/sglang) framework to run on SageMaker AI endpoints. SGLang is a serving framework for large language models that provides state-of-the-art performance, including a fast backend runtime for efficient serving with RadixAttention, extensive model support, and an active open-source community. For more information refer to [https://docs.sglang.ai/index.html](https://docs.sglang.ai/index.html) and [https://github.com/sgl-project/sglang](https://github.com/sgl-project/sglang).

By using SGLang and building a custom Docker container, you can run advanced AI models like the [Qwen2.5-VL-32B-Instruct](https://huggingface.co/Qwen/Qwen2.5-VL-32B-Instruct)  model on a SageMaker AI endpoint.

### Set up Environment

In [ ]:
%%capture --no-stderr

!pip install -U pip
!pip install -U "sagemaker>=2.237.3"

### Create SageMaker AI endpoint for EXAONE Deep 7.8B model

In [ ]:
import boto3

region = boto3.Session().region_name
region

In [ ]:
import boto3
from typing import List


def get_cfn_outputs(stackname: str, region_name: str='us-east-1') -> List:
    cfn = boto3.client('cloudformation', region_name=region_name)
    outputs = {}
    for output in cfn.describe_stacks(StackName=stackname)['Stacks'][0]['Outputs']:
        outputs[output['OutputKey']] = output['OutputValue']
    return outputs

In [ ]:
CFN_STACK_NAME = "QwenVLSGLangRealtimeEndpointStack"
cfn_stack_outputs = get_cfn_outputs(CFN_STACK_NAME, region)

endpoint_name = cfn_stack_outputs['EndpointName']
model_name = cfn_stack_outputs['ModelName']

model_name, endpoint_name

In [ ]:
from sagemaker import Predictor
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer


predictor = Predictor(
    endpoint_name=endpoint_name,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer()
)

### Invoke endpoint with SageMaker Python SDK

In [ ]:
%%time

# https://github.com/sgl-project/sglang/blob/v0.4.4.post3/python/sglang/srt/conversation.py#L499
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image_url",
                "image_url": {
                    "url": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg"
                },
            },
            {"type": "text", "text": "Describe this image."},
        ],
    }
]

response = predictor.predict({
    'model': 'default',
    'messages': messages,
    'temperature': 0.6,
    'max_new_tokens': 128,
    'do_sample': True,
    'top_p': 0.95,
})

print(response['choices'][0]['message']['content'])

The image depicts a serene beach scene with a person and a dog. The person is sitting on the sandy beach, facing the ocean. They are dressed in a plaid shirt and black pants, and they have their legs crossed. The dog, a light-colored breed possibly a Labrador Retriever, is sitting beside them, facing the person. The dog has its paw raised, as if it is giving a paw or pawing at the person's hand. The ocean waves are gently crashing onto the shore, and the sky is clear with a soft, warm light suggesting either sunrise or sunset. The overall atmosphere is calm and peaceful.
CPU times: user 3.8 ms, sys: 0 ns, total: 3.8 ms
Wall time: 7.29 s


In [ ]:
import base64

def encode_image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        encoded_image = base64.b64encode(image_file.read())
        return f"data:image;base64,{encoded_image.decode('utf-8')}"

In [ ]:
%%time

base64_image = encode_image_to_base64("./examples/image1.png")

# https://github.com/sgl-project/sglang/blob/v0.4.4.post3/python/sglang/srt/conversation.py#L499
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image_url",
                "image_url": {
                    "url": base64_image
                },
            },
            {"type": "text", "text": "Extract all text from the image and separate each line or text segment with a newline character."},
        ],
    }
]

response = predictor.predict({
    'model': 'default',
    'messages': messages,
    'temperature': 0.6,
    'max_new_tokens': 128,
    'do_sample': True,
    'top_p': 0.95,
})

print(response['choices'][0]['message']['content'])

- 개봉선 -
자이스 렌즈 와이프 렌즈 표면의 먼지와 얼룩을
흔적 없이 부드럽게 닦아주는 일회용 타슈.
안전기준 안전확인대상생활화학제품 확인 표시사항
제 FB21-02-0531호
품목명: 자이스 렌즈 와이프
제조자, 제조조국: 프로스베니아이앤씨(Prosbena, Inc.)
제조연월: 제품 하단 LOT 번호 앞 네 자리 참조
수입자, 주소, 연락처: 칸자이스비진코리아, 중국
서울시 송파구 법원로 135, 1201호(02-2252-1001)
www.zeiss.com/Cleaning
10116-94-004
Produced in China
CPU times: user 5.85 ms, sys: 227 μs, total: 6.08 ms
Wall time: 3.86 s


### Invoke endpoint with boto3

Note that you can also invoke the endpoint with boto3. If you have an existing endpoint, you don't need to recreate the predictor and can follow below example to invoke the endpoint with an endpoint name.

In [ ]:
import boto3
import json

sagemaker_runtime = boto3.client('sagemaker-runtime', region_name=region)
endpoint_name = predictor.endpoint_name # you can manually set the endpoint name with an existing endpoint

In [ ]:
%%time

# https://github.com/sgl-project/sglang/blob/v0.4.4.post3/python/sglang/srt/conversation.py#L499
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image_url",
                "image_url": {
                    "url": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg"
                },
            },
            {"type": "text", "text": "Describe this image."},
        ],
    }
]

prompt = {
    'model': 'default',
    'messages': messages,
    'temperature': 0.6,
    'max_new_tokens': 128,
    'do_sample': True,
    'top_p': 0.95
}

response = sagemaker_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Body=json.dumps(prompt)
)

response_dict = json.loads(response['Body'].read().decode("utf-8"))
response_content = response_dict['choices'][0]['message']['content']
print(response_content)

The image depicts a serene beach scene during what appears to be either sunrise or sunset. The sky is a soft gradient of warm colors, transitioning from a light blue at the top to a deeper orange near the horizon. The ocean waves are gently crashing onto the shore, creating small, frothy white foam. 

On the sandy beach, there is a person and a dog. The person is sitting on the sand, facing the dog, and appears to be interacting with it. The dog, which is likely a Labrador Retriever, is sitting on the sand with its back to the camera, facing the person. The dog's front paws are extended towards the person's hand, suggesting a friendly and playful interaction. The person is wearing a plaid shirt and dark pants, and they have a relaxed posture, indicating a moment of connection with the dog.

The overall atmosphere of the image is calm and joyful, capturing a heartwarming moment between a person and their dog at the beach. The lighting and colors contribute to a sense of warmth and tranq

In [ ]:
%%time

base64_image = encode_image_to_base64("./examples/image1.png")

# https://github.com/sgl-project/sglang/blob/v0.4.4.post3/python/sglang/srt/conversation.py#L499
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image_url",
                "image_url": {
                    "url": base64_image
                },
            },
            {"type": "text", "text": "Extract all text from the image and separate each line or text segment with a newline character."},
        ],
    }
]

prompt = {
    'model': 'default',
    'messages': messages,
    'temperature': 0.6,
    'max_new_tokens': 128,
    'do_sample': True,
    'top_p': 0.95
}

response = sagemaker_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Body=json.dumps(prompt)
)

response_dict = json.loads(response['Body'].read().decode("utf-8"))
response_content = response_dict['choices'][0]['message']['content']
print(response_content)

- 개봉전
자이스 렌즈 와이프 렌즈 표면의 먼지와 얼룩을
흔적 없이 부드럽게 닦아주는 일회용 타슈.
- 안전기준
안전확인대상생활화학제품
확인
표시사항
- 신고번호: 제 FB21-02-0531호
품목: 제거제
제품명: 자이스 렌즈 와이프
주요물질: 정제수, 2-프로판올
제조연월: 제품 하단 LOT 번호 앞 네 자리 참조
제조자, 제조국: 프로스베니아이앤씨(Prosben, Inc.), 중국
수입자, 주소, 연락처: 칸자이스비진코리아, 서울시 송파구 법원로 135, 1201호(02-2252-1001)
CPU times: user 4.7 ms, sys: 619 μs, total: 5.32 ms
Wall time: 3.97 s


### Clean up the environment

Make sure to delete the endpoint and other artifacts that were created to avoid unnecessary cost. You can also go to SageMaker AI console to delete all the resources created in this example.

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()

### References

- [Qwen2.5-VL-32B-Instruct Model Card](https://huggingface.co/Qwen/Qwen2.5-VL-32B-Instruct)
- [SGLang Documentation](https://docs.sglang.ai/index.html) - a fast serving framework for large language models and vision language models
- [sagemaker-genai-hosting-examples/Deepseek/SGLang-Deepseek/deepseek-r1-llama-70b-sglang.ipynb](https://github.com/aws-samples/sagemaker-genai-hosting-examples/blob/main/Deepseek/SGLang-Deepseek/deepseek-r1-llama-70b-sglang.ipynb)